In [2]:
!pip install -qqq langchain-core langgraph>0.2.27

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [5]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.0 MB/s eta 0:00:00


In [6]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

··········


In [14]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Anand")])

AIMessage(content="Hello Anand! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 16, 'total_tokens': 43, 'completion_time': 0.0225, 'prompt_time': 0.002251686, 'queue_time': 0.011897272, 'total_time': 0.024751686}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-8952199f-fec0-4ba7-a765-2448a6fa8bd8-0', usage_metadata={'input_tokens': 16, 'output_tokens': 27, 'total_tokens': 43})

In [15]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have any information about your personal identity, including your name. I'm a new conversation each time you interact with me, and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to chat with you and learn more about you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 15, 'total_tokens': 94, 'completion_time': 0.065833333, 'prompt_time': 0.002252118, 'queue_time': 0.012837902000000002, 'total_time': 0.068085451}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-007f6deb-f21c-48d3-b825-ff0aa6de5b9b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 79, 'total_tokens': 94})

In [16]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Anand"),
        AIMessage(content="Hello Anand! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='According to our conversation, your name is Anand!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 42, 'total_tokens': 54, 'completion_time': 0.01, 'prompt_time': 0.005093808, 'queue_time': 0.00953658, 'total_time': 0.015093808}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-441b39f0-b579-4765-a59a-5fa1a1946dab-0', usage_metadata={'input_tokens': 42, 'output_tokens': 12, 'total_tokens': 54})

In [17]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [11]:
config = {"configurable": {"thread_id": "abc123"}}

In [12]:
query = "Hi! I'm Anand."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Anand! Nice to meet you! Is there something I can help you with or would you like to chat?


In [19]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize again, but as a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to introduce yourself, I'd be happy to chat with you!


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)